In [1]:
import syft as sy
import torch
from tools import models, datasets
import numpy as np
import pandas as pd
import opacus

In [ ]:
train_ds, test_ds, val_ds = datasets.Loader.load_MedNIST(sample_size=0.02, test_size=0.1, val_size=0.1)
train_data, train_labels = train_ds.as_tensor()
test_data, test_labels = test_ds.as_tensor()
val_data, val_labels = val_ds.as_tensor()

In [ ]:
# Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = True # Whether or not this run should be tracked in the results.csv
DP = True # Whether or not Differential Privacy should be applied

# Parameters for training and Differential Privacy
length = len(train_data)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

BATCH_SIZE = 50
EPOCHS = 20
LEARNING_RATE = 0.003 if DP else 0.0001

DELTA = 0.001 # Set to be less then the inverse of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)
NOISE_MULTIPLIER = 2.0 # The amount of noise sampled and added to the average of the gradients in a batch (from https://opacus.ai/tutorials/building_image_classifier)
MAX_GRAD_NORM = 1.2 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step (from https://opacus.ai/tutorials/building_image_classifier)

# Getting model
model = models.Deep2DNet(torch)

# Setting device to train on
cuda_available = torch.cuda.is_available()
if cuda_available:
    device = torch.device('cuda:0')
    model.cuda(device)
else:
    device = torch.device('cpu')
    model.cpu()

# Optimizer and Loss Function
params = model.parameters()
optim = torch.optim.Adam(params=params, lr=LEARNING_RATE) # without DP: 0.0001 // with DP: 0.002-0.003
loss_function = torch.nn.CrossEntropyLoss()

# Setting up Differential Privacy Engine
if DP:
    privacy_engine = opacus.privacy_engine.PrivacyEngine(
        model.real_module, sample_rate=SAMPLE_RATE,
        noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
    )
    privacy_engine.attach(optim)
else:
    privacy_engine = None

In [ ]:
import time

def train(batch_size, epochs, model, 
          torch_ref, optim, loss_function, 
          train_data, train_labels, test_data, 
          test_labels, input_shape, device, privacy_engine=None):
    
    # Variables to track
    losses = [] # Training losses per batch per epoch
    test_accs = []
    test_losses = [] # Test losses per epoch
    epsilons = [] 
    alphas = []
    epoch_times = [] # Training times for each epoch
    best_acc_loss = (0, 0)
    best_model = None
    
    # Divide dataset into batches (sadly remote DataLoaders aren't yet a thing in pysyft)
    length = len(train_data)
    
    if length % batch_size != 0:
        cut_data = train_data[:length - length % batch_size]
        cut_labels = train_labels[:length - length % batch_size]
        
    shape = [-1, batch_size]
    shape.extend(input_shape)
    
    batch_data = cut_data.view(shape)
    batch_labels = cut_labels.view(-1, batch_size)
    
    # Prepare indices for randomization of order for each epoch
    indices = np.arange(int(length / batch_size))
    
    for epoch in range(epochs):
        epoch_start = time.time()
        epoch_loss = []
        
        model.train()
        
        np.random.shuffle(indices)
        
        print(f'###### Epoch {epoch + 1} ######')
        for i in indices:
            optim.zero_grad()
            
            output = model(batch_data[int(i)].to(device))
            
            loss = loss_function(output, batch_labels[int(i)].to(device))
            loss_item = loss.item()
            
            if model.is_local:
                loss_value = loss_item
            else:
                loss_value = loss_item.get(reason="To evaluate training progress", request_block=True, timeout_secs=5)
            print(f'Training Loss: {loss_value}')
            epoch_loss.append(loss_value)
        
            loss.backward()
            optim.step()
        
        # Checking our privacy budget
        if privacy_engine is not None:
            epsilon_tuple = privacy_engine.get_privacy_spent(DELTA)
            epsilon_ptr = epsilon_tuple[0].resolve_pointer_type()
            best_alpha_ptr = epsilon_tuple[1].resolve_pointer_type()

            epsilon = epsilon_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )
            best_alpha = best_alpha_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )
            if epsilon is None:
                epsilon = float("-inf")
            if best_alpha is None:
                best_alpha = float("-inf")
            print(
                f"(ε = {epsilon:.2f}, δ = {DELTA}) for α = {best_alpha}"
            )
            epsilons.append(epsilon)
            alphas.append(best_alpha)
    
        test_acc, test_loss = test(model, loss_function, torch_ref, test_data, test_labels, device)
        print(f'Test Accuracy: {test_acc} ---- Test Loss: {test_loss}')
        
        epoch_end = time.time()
        print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
        
        losses.append(epoch_loss)
        epoch_times.append(int(epoch_end - epoch_start))
        test_accs.append(test_acc)
        test_losses.append(test_loss)
        
    return losses, test_accs, test_losses, epsilons, alphas, epoch_times
                   
            
def test(model, loss_function, torch_ref, data, labels, device):
    model.eval()
    
    data = data.to(device)
    labels = labels.to(device)
    length = len(data)
    
    with torch_ref.no_grad():
        output = model(data)
        test_loss = loss_function(output, labels)
        prediction = output.argmax(dim=1)
        total = prediction.eq(labels).sum().item()
        
    acc_ptr = total / length
    if model.is_local:
        acc = acc_ptr
        loss = test_loss.item()
    else:
        acc = acc_ptr.get(reason="To evaluate training progress", request_block=True, timeout_secs=5)
        loss = test_loss.item().get(reason="To evaluate training progress", request_block=True, timeout_secs=5)

    return acc, loss

In [ ]:
losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, 
                                                                      model, torch,
                                                                      optim, loss_function, 
                                                                      train_data, train_labels, 
                                                                      test_data, test_labels, 
                                                                      [1, 64, 64], device, privacy_engine)

In [ ]:
# Tracking all interesting variables and results in .csv file
if TRACKING:
    d = {
        'model': MODEL,
        'dataset': DATASET,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'train_sample_size': SAMPLE_SIZE,
        'test_sample_size': len(test_data_ptr),
        'val_sample_size': len(val_data),
        'delta': DELTA,
        'noise_multiplier': NOISE_MULTIPLIER,
        'max_grad_norm': MAX_GRAD_NORM,
        'dp_used': DP,
        'epsilons': epsilons,
        'alphas': alphas,
        'train_losses': losses,
        'test_accs': test_accs,
        'test_losses': test_losses,
        'val_acc': 0.0, #val_acc,
        'val_loss': 0.0, #val_loss,
        'epoch_times': epoch_times
    }      
    df = pd.read_csv('./Results/1DS.csv')
    df = df.append(d, ignore_index=True)
    df.to_csv('./Results/1DS.csv')